In [2]:
import subprocess
import json

# Global variables to track processing state and store strings
process_file = False
original_string = "stream"
ffmpeg_command = " "

def check_media_file(file_path):
    # Initial ffmpeg check
    if initial_ffmpeg_check(file_path) == "Success":
        result = full_workflow(file_path)


def initial_ffmpeg_check(file_path):
    ffmpeg_command = f'ffmpeg -v error -i "{file_path}" -f null -'
    try:
        result = subprocess.run(ffmpeg_command, shell=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        if result.stdout or result.stderr:
            #print (file_path + " file is invalid")          
            return "Failure"
        else:
            #print (file_path + " file is valid") 
            return "Success"
    except subprocess.CalledProcessError as e:
        #print(result.stdout)
        return "Error"


def full_workflow(file_path):
    ffprobe_command = f'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout,format=nb_streams -of json "{file_path}"'
    
    try:
        result = subprocess.run(ffprobe_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        stream_info = json.loads(result.stdout)
        streams_count = stream_info['format']['nb_streams']
        for i in range(streams_count):
            process_stream(stream_info['streams'][i], i)

        ffmpeg_string = 'ffmpeg -hide_banner -loglevel 16 -stats -stats_period 10 -i "' + file_path + '"' + ffmpeg_command + ' "' + file_path + '"'
        print ("original_string is: " + original_string)
        print ("ffmpeg_command is: " + ffmpeg_string)
        print ("process_file is:" + str(process_file))
    except subprocess.CalledProcessError as e:
        print(result.stdout)

def process_stream(stream, index):
    global original_string
    codec_type = stream.get('codec_type')
    #print ("codec_type is: " + codec_type)
    codec_name = stream.get('codec_name')
    channel_layout = stream.get('channel_layout')

    original_string = original_string + " 0:" + str(index) + " " + codec_type + ":" + codec_name
    
    if codec_type == 'video':
        process_video_stream(codec_name, index)
    elif codec_type == 'audio':
        process_audio_stream(codec_name, channel_layout, index)
    elif codec_type == 'subtitle':
        process_subtitle_stream(codec_name, index)
    elif codec_type == 'attachment':
        process_attachment_stream(codec_name, index)
    else:
        print(f"Other stream type: codec_name={codec_name}, codec_type={codec_type}")

def process_video_stream(codec_name, index):
    global process_file, ffmpeg_command
    #print ("codec_name is: " + codec_name)
    if codec_name == 'av1':
        #print(f"Video stream {index}: codec_name=av1")
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:v stuff'
        #print (process_file)
        #print (ffmpeg_command)
    else:
        #print(f"Video stream {index}: codec_name={codec_name}")
        ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:v copy'
        process_file = True
        #print (process_file)
        #print (ffmpeg_command)

def process_audio_stream(codec_name, channel_layout, index):
    global process_file, ffmpeg_command
    #print(f"Audio stream: codec_name={codec_name}, channel_layout={channel_layout}")
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:a copy'
    
def process_subtitle_stream(codec_name, index):
    global process_file, ffmpeg_command
    #print(f"Subtitle stream: codec_name={codec_name}")
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:s copy'

def process_attachment_stream(codec_name, index):
    global process_file, ffmpeg_command
    #print(f"Attachment stream: codec_name={codec_name}")
    ffmpeg_command = ffmpeg_command + ' -map 0:' + str(index) + ' -c:t copy'


hhelo


In [31]:
import subprocess
import json













In [84]:
import subprocess
import json


def check_video_stream(codec_name, encoding_decision):
    # Checks the video stream from check_codecs to determine if the stream needs encoding
    print (codec_name)
    desired_video_codec = 'av1'
    if codec_name != desired_video_codec:
        print ('Container requires encoding')
        encoding_decision = True
    return encoding_decision

def check_audio_stream(codec_name, encoding_decision):
    # Checks the audio stream from check_codecs to determine if the stream needs encoding
    print (codec_name)
    # This will be populated at a later date
    #desired_audio_codec = 'aac'
    #if codec_name != desired_video_codec:
    #    encoding_decision = True
    return encoding_decision
    
def check_subtitle_stream(codec_name, encoding_decision):
    # Checks the subtitle stream from check_codecs to determine if the stream needs encoding
    print (codec_name)
    # This will be populated at a later date
    #desired_subtitle_codec = 'srt'
    #if codec_name != desired_subtitle_codec:
    #    encoding_decision = True
    return encoding_decision

def check_attachmeent_stream(codec_name, encoding_decision):
    # Checks the attachment stream from check_codecs to determine if the stream needs encoding
    print (codec_name)
    # This will be populated at a later date
    #desired_attachment_codec = '???'
    #if codec_name != desired_attachment_codec:
    #    encoding_decision = True
    return encoding_decision

def check_codecs(stream_info, encoding_decision):
    # Loops through the streams in stream_info from requires_encoding, then
    # calls functions to determine if the steam needs encoding based on stream type conditions 
    streams_count = stream_info['format']['nb_streams']
    print ('There are : ' + str(streams_count) + ' streams')
    for i in range (0,streams_count):
        codec_type = stream_info['streams'][i]['codec_type'] 
        codec_name = stream_info['streams'][i]['codec_name'] 
        if codec_type == 'video':
            check_video_stream(codec_name, encoding_decision, i)
        elif codec_type == 'audio':
            check_audio_stream(codec_name, encoding_decision, i)
        elif codec_type == 'subtitle':
            check_subtitle_stream(codec_name, encoding_decision, i)
        elif codec_type == 'attachment':
            check_attachmeent_stream(codec_name, encoding_decision, i)        
    return encoding_decision

def check_container_type(stream_info, encoding_decision):
    # Desired container is MKV so we check for that, and pass True for all other container types
    format_name = stream_info['format'].get('format_name')
    print ('format is: ' + format_name)
    if format_name != 'matroska,webm':
        encoding_decision = True
    print ('>>>check_container_type<<<  Container is: ' + format_name + ' so, encoding_decision is: ' + str(encoding_decision))
    return encoding_decision
    

def ffprobe_function(file_path):
    # Subprocess call to ffprobe to retrieve video info in JSON format
    ffprobe_command = f'ffprobe -loglevel quiet -show_entries format:stream=index,stream,codec_type,codec_name,channel_layout,format=nb_streams -of json "{file_path}"'
    result = subprocess.run(ffprobe_command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stream_info = json.loads(result.stdout)
    return stream_info


def requires_encoding(file_path):
    encoding_decision = False
    stream_info = ffprobe_function(file_path)
    ffmpeg_command = str()
    if check_container_type(stream_info, encoding_decision) == True or check_codecs(stream_info, encoding_decision) == True:
        print ('file does not need encoding')
        
    else:
        print ('file needs encoding')
    print (encoding_decision)





In [85]:
file_path = 'C:\\Users\\cwest\\Videos\\BoilestDev\\test22.mkv'
requires_encoding(file_path)

format is: matroska,webm
>>>check_container_type<<<  Container is: matroska,webm so, encoding_decision is: False
There are : 10 streams
hevc
Container requires encoding
aac
ass
ttf
ttf
ttf
ttf
ttf
ttf
ttf
file needs encoding
False
